# Статистические тесты

In [50]:
import numpy as np
import pandas as pd
from scipy import stats

In [51]:
np.random.seed(13)
random_normal = np.random.normal(5, 2, 100)
random_bin = np.random.choice([0, 1], size=(100,), p=[0.8, 0.2])

In [52]:
# Проверим гипотезу, что среднее выборки, сгенерированной из нормального распределения, равно 0:
stats.ttest_1samp(random_normal, 0.0)

Ttest_1sampResult(statistic=27.315846581411247, pvalue=6.762242033211871e-48)

По результату мы видим, что p_value= 6.762242033211871e-48, соответственно, гипотеза о равенстве среднего нулю отвергается на уровне значимости 0.01 (вообще говоря, почти на любом уровне значимости).

In [53]:
# Теперь проверим против 5.0:
stats.ttest_1samp(random_normal, 5.0)

Ttest_1sampResult(statistic=0.6236095710595042, pvalue=0.5343182132984923)

Мы видим, что гипотеза о равенстве 5 принимается с p_value=0.53.

In [54]:
# Проведём тест на равенство доли единиц во второй выборке 0.5:
stats.binom_test(x=[sum(random_bin), len(random_bin) - sum(random_bin)], p=0.5)

1.1159089057251951e-09

Заметим, что x = [число позитивных действий, число фейлов]. Видим, что гипотеза отвергается практически на любом уровне значимости с p_value=1.115908905725195e-09.

In [55]:
stats.binom_test(x=[sum(random_bin), len(random_bin) - sum(random_bin)], p=0.2)

1.0

Гипотеза не отвергается.

А ТЕПЕРЬ — ДВУХВЫБОРОЧНЫЙ ТЕСТ!

In [56]:
random_normal_5 = np.random.normal(5, 2, 100)
random_normal_false = np.random.normal(7, 2, 100)

In [57]:
print(stats.ttest_ind(random_normal_5, random_normal))
print(stats.ttest_ind(random_normal_false, random_normal))

Ttest_indResult(statistic=-1.1097766930134212, pvalue=0.26844127144491964)
Ttest_indResult(statistic=7.56418694452888, pvalue=1.4333446457467145e-12)


In [58]:
# тест на равенство пропорций (долей)
from statsmodels.stats.proportion import proportions_ztest

random_bin_2 = np.random.choice([0, 1], size=(100,), p=[0.8, 0.2])
random_bin_false = np.random.choice([0, 1], size=(100,), p=[0.6, 0.4])

print(proportions_ztest(count=[sum(random_bin), sum(random_bin_2)], nobs=[len(random_bin), len(random_bin_2)]))
print(proportions_ztest(count=[sum(random_bin), sum(random_bin_false)], nobs=[len(random_bin), len(random_bin_false)]))

(-0.5163595320566151, 0.6056033242983083)
(-2.5197631533948477, 0.011743382301172597)


Первое число — значение статистики, второе — p_value. Видим, что результаты совпадают с ожидаемыми.

Обратим внимание, что в сам тест мы передаём не выборки, а только число положительных событий и общее число событий.

## Тесты на нормальность

In [59]:
np.random.seed(13)
random_normal = np.random.normal(5, 2, 100)
random_uniform = np.random.uniform(0, 1, 100)

In [60]:
print(stats.shapiro(random_normal))
print(stats.shapiro(random_uniform))

ShapiroResult(statistic=0.9897646903991699, pvalue=0.6455868482589722)
ShapiroResult(statistic=0.9223209619522095, pvalue=1.872835127869621e-05)


Видим, что во втором случае нулевая гипотеза (нормальность распределения, из которого приходят точки) отвергается практически на любом уровне значимости.

In [61]:
#  Для теста Колмогорова-Смирнова необходимо передавать распределение, относительно которого происходит сравнение.
print(stats.kstest(random_normal, 'norm'))
print(stats.kstest(random_uniform, 'norm'))

KstestResult(statistic=0.933371471007256, pvalue=4.646377515900965e-118)
KstestResult(statistic=0.5039276991503323, pvalue=4.926235881910341e-24)


Мы видим, что в обоих случаях гипотеза о нормальности распределения отвергается. Почему? Потому что происходит сравнение с нормальным распределением со средним 0 и дисперсией 1. Проверим это, нормализовав входное множество точек.

In [62]:
stats.kstest((random_normal - np.mean(random_normal)) / np.std(random_normal), 'norm')

KstestResult(statistic=0.06562034353090856, pvalue=0.7572142037683822)

In [63]:
np.random.seed(2)
random_normal = np.random.normal(2, 4, 10)
print(stats.shapiro(random_normal))
print(stats.kstest(random_normal, 'norm'))
print(stats.kstest((random_normal - np.mean(random_normal)) / np.std(random_normal), 'norm'))

ShapiroResult(statistic=0.9548439383506775, pvalue=0.7258638739585876)
KstestResult(statistic=0.514185695796664, pvalue=0.005575825333255893)
KstestResult(statistic=0.17907654022324243, pvalue=0.8514658527102467)


## Множественная проверка Гипотез

In [64]:
from statsmodels.stats.multitest import multipletests

In [65]:
np.random.seed(2)
pvals = np.random.uniform(0, 0.3, 100)
#Скорректируем поправкой Холма:
pvals_corrected = multipletests(pvals, method="holm")

## Практика

In [66]:
df_train = pd.read_csv("data/train.csv", index_col="PassengerId")
df_test = pd.read_csv("data/test.csv", index_col="PassengerId")

In [67]:
# Правда ли, что мужчины реже выживали, чем женщины?
male_survived = df_train[df_train['Sex']=='male']['Survived'].sum()
male_num = len(df_train[df_train['Sex']=='male']['Survived'])
female_survived = df_train[df_train['Sex']=='female']['Survived'].sum()
female_num = len(df_train[df_train['Sex']=='female']['Survived'])
proportions_ztest(count=[male_survived, female_survived], nobs=[male_num , female_num])

(-16.218833930670097, 3.7117477701134797e-59)

Мы видим, что есть статистически значимое различие между числом выживших мужчин и женщин. Но наша цель не в том, чтобы зафиксировать наличие различий в уровне выживаемости, а выяснить, выживали ли мужчины реже. У нас есть два варианта: просто посмотреть, где больше выживших с учётом проверенной гипотезы, или провести одностороннюю проверку гипотезы. Последний вариант вполне подходит:

In [68]:
proportions_ztest(count=[male_survived, female_survived], nobs=[male_num , female_num], alternative='smaller')

(-16.218833930670097, 1.8558738850567398e-59)

In [69]:
# Верно ли, что погибшие в среднем были старше, чем выжившие?
df_cleared = df_train.dropna()
survived_ages = df_cleared[df_cleared['Survived']==1]['Age'].values
died_ages = df_cleared[df_cleared['Survived']!=1]['Age'].values
stats.ttest_ind(died_ages, survived_ages, alternative="greater")

Ttest_indResult(statistic=3.53435125095576, pvalue=0.0002594751653940835)

Действительно, средний возраст погибших больше, чем средний возраст выживших.

Разобьём его на декады и проверим, верно ли, что выживаемость между различными возрастными группами различна. При этом помним, что мы с вами проводим множественную проверку гипотез:

In [70]:
df_cleared['age_group'] = df_cleared['Age'].apply(lambda x: x//10)
p_vals = []
coeffs = []
age_groups = df_cleared['age_group'].unique()
for i in range(len(age_groups)):
    age_group_1_sum = df_cleared[df_cleared['age_group']==age_groups[i]]['Survived'].sum()
    age_group_1_count = len(df_cleared[df_cleared['age_group']==age_groups[i]])
    for j in range(i+1, len(age_groups)):
        age_group_2_sum = df_cleared[df_cleared['age_group']==age_groups[j]]['Survived'].sum()
        age_group_2_count = len(df_cleared[df_cleared['age_group']==age_groups[j]])
        p_value = proportions_ztest(count=[age_group_1_sum, age_group_2_sum], nobs=[age_group_1_count, age_group_2_count])[1]
        p_vals.append(p_value)
        coeffs.append(age_groups[j])
p_vals_corrected = multipletests(p_vals, method="bonferroni")[1]

for i, pval in enumerate(p_vals_corrected):
    if pval < 0.05:
        print(coeffs[i])

C:\Users\Пользователь\AppData\Local\Temp\ipykernel_6360\2843580256.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleared['age_group'] = df_cleared['Age'].apply(lambda x: x//10)


Таким образом, в итоговую модель целесообразно включать пол и возраст. А добавление возрастных групп, напротив, выглядит сомнительно.

In [91]:
# Тест на равенство среднего возраста в тренировочной и тестовой выборках
H0 = 'Средний возваст между выборками равен'
H1 = 'Средний возвраст между выборками не равен'
a = 0.05 # Уровень значимости
_, p_value = stats.ttest_ind(a = df_train['Age'].dropna().values, b = df_test['Age'].dropna().values)
if p_value < a:
    print(f'p_value = {p_value}. Отвергаем H0:', H1)
else:
    print(f'p_value = {p_value}. Принимаем H0:', H0)

p_value = 0.5494539128955762. Принимаем H0: Средний возваст между выборками равен


In [92]:
df_train.head(2)

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C


In [98]:
# Тест на равенство доли мужчин в тренировочной и тестовой выборках
H0 = 'Доля мужчин в тренировочной и тестовой выборках равна'
H1 = 'Доля мужчин в тренировочной и тестовой выборках равна'
a = 0.05 # Уровень значимости
_, p_value = proportions_ztest(count=[df_train[df_train['Sex'] == 'male']['Sex'].count(), df_test[df_test['Sex'] == 'male']['Sex'].count()],
                               nobs=[df_train['Sex'].count(), df_test['Sex'].count()])
if p_value < a:
    print(f'p_value = {p_value}. Отвергаем H0:', H1)
else:
    print(f'p_value = {p_value}. Принимаем H0:', H0)

p_value = 0.6925640325233131. Принимаем H0: Доля мужчин в тренировочной и тестовой выборках равна
